In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spp.convex_sets import Singleton, Polyhedron
from spp.convex_functions import TwoNorm
from spp.graph import GraphOfConvexSets
from spp.shortest_path import ShortestPathProblem

In [ ]:
# vertices
sets = [
    Singleton([0, 0]),
    Singleton([0, 2]),
    Singleton([0, -2]),
    Polyhedron.from_vertices([[2, -2.1], [4, -2.1], [4, 2.1], [2, 2.1]]),
    Singleton([5, 0])
]
sets[3]._center = np.array([3, 0])
vertices = ['s', '1', '2', '3', 't']

# add convex sets to the graph
G = GraphOfConvexSets()
G.add_sets(sets, vertices)
G.set_source('s')
G.set_target('t')

# edges
H = np.hstack((np.eye(2), -np.eye(2)))
l = TwoNorm(H)
edges = {
    's': ('1', '2'),
    '1': ('3',),
    '2': ('3',),
    '3': ('t',),
}
for u, vs in edges.items():
    for v in vs:
        G.add_edge(u, v, l)

# Plot solution of MICP

In [ ]:
spp = ShortestPathProblem(G, relaxation=0)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure(figsize=(4,4))
G.draw_sets()
G.draw_edges()

plt.text(0.15, 0, r'$X_s$', ha='left', va='center')
plt.text(0.4, 1.9, r'$X_1$', ha='center', va='bottom')
plt.text(0.4, -1.9, r'$X_2$', ha='center', va='top')
plt.text(3, 0.1, r'$X_3$', ha='left', va='bottom')
plt.text(5, 0.1, r'$X_t$', ha='center', va='bottom')

kwargs = {'marker': 'o', 'markeredgecolor': 'k', 'markerfacecolor': 'w'}
G.draw_path(phi, x, color='b', linestyle='--')

plt.grid()
# plt.savefig('symmetry_mip.pdf', bbox_inches='tight')

# Plot solution of convex relaxation

In [ ]:
spp = ShortestPathProblem(G, relaxation=1)
sol = spp.solve()
phi = sol.primal.phi
x = sol.primal.x
y = sol.primal.y
z = sol.primal.z

print('Cost:', sol.cost)
print('\nFlows:')
for k, edge in enumerate(G.edges):
    flow = round(abs(phi[k]), 4)
    print(edge, flow)

In [ ]:
plt.figure(figsize=(4,4))
G.draw_sets()

plt.text(.1, 0,
         r'$\bar y_{(s,1)} = \bar y_{(s,2)}$',
         ha='left', va='center')
plt.text(.1, 1.9,
         r'$\bar z_{(s,1)} = \bar y_{(1,3)}$',
         ha='left', va='top')
plt.text(.1, -1.9,
         r'$\bar z_{(s,2)} = \bar y_{(2,3)}$',
         ha='left', va='bottom')
plt.text(3, 1.9,
         r'$\bar z_{(1,3)}$',
         ha='center', va='top')
plt.text(3, -1.9,
         r'$\bar z_{(2,3)}$',
         ha='center', va='bottom')
plt.text(3, 0.1,
         r'$\bar y_{(3,t)}$',
         ha='center', va='bottom')
plt.text(5, .1,
         r'$\bar z_{(3,t)}$',
         ha='right', va='bottom')

plt.text(.1, 1, r'$0.5$', ha='left', va='center', c='r')
plt.text(.1, -1, r'$0.5$', ha='left', va='center', c='r')
plt.text(1, 1.99, r'$0.5$', ha='center', va='bottom', c='r')
plt.text(1, -2.05, r'$0.5$', ha='center', va='top', c='r')
plt.text(3.95, 0, r'$1.0$', ha='right', va='bottom', c='r')

y3t = y[4] / phi[4]
z13 = z[2] / phi[2]
z23 = z[3] / phi[3]

plt.plot([0, 0], [0, 2], 'b--', **kwargs)
plt.plot([0, 0], [0, -2], 'b--', **kwargs)
plt.plot([0, z13[0]], [2, z13[1]], 'b--', **kwargs)
plt.plot([0, z23[0]], [-2, z23[1]], 'b--', **kwargs)
plt.plot([y3t[0], 5], [y3t[1], 0], 'b--', **kwargs)

plt.grid()
# plt.savefig('symmetry_relaxation.pdf', bbox_inches='tight')